### Include registration modules

In [16]:
import copy

from functools import partial
from pathlib import Path
from time import time
from typing import Callable

import numpy as np
import open3d.geometry as geom
import open3d.pipelines.registration as reg
import open3d.visualization as vis
import open3d.utility as util

from result import Ok

from benthoscan.runtime import Environment, load_environment

from benthoscan.registration import (
    MultiTargetIndex,
    PointCloud,
    PointCloudLoader,
    read_point_cloud,
    downsample_point_cloud,
    estimate_point_cloud_normals,
    generate_cascade_indices,
)

from benthoscan.registration import (
    ExtendedRegistrationResult,
    register_point_cloud_fphp_fast,
    register_point_cloud_fphp_ransac,
    register_point_cloud_icp,
    register_point_cloud_graph,
)

# from benthoscan.tasks.registration import RegistrationTaskConfig

from benthoscan.utils.log import logger

### Load environment and configure data loaders

In [17]:
environment: Environment = load_environment()

DATA_DIR: Path = Path("/home/martin/dev/benthoscan/.cache")

point_cloud_files: dict = {
    0: DATA_DIR / Path("qdc5ghs3_20100430_024508.ply"),
    1: DATA_DIR / Path("qdc5ghs3_20120501_033336.ply"),
    2: DATA_DIR / Path("qdc5ghs3_20130405_103429.ply"),
    3: DATA_DIR / Path("qdc5ghs3_20210315_230947.ply"),
}

loaders: dict = {
    key: partial(read_point_cloud, path=path) for key, path in point_cloud_files.items()
}

count = len(loaders)
if count < 2:
    logger.error(f"invalid number of point clouds for registration: {count}")

### Configure preprocessor and registrator

In [29]:
# NOTE: Parameters - move to config
VOXEL_SIZE: float = 0.20

# FPFH feature parameters
FEATURE_RADIUS: float = 2.00  # NOTE: 2.00
FEATURE_NEIGHBOURS: int = 500

# Validation and estimation
CORRESPONDENCE_DISTANCE: float = 0.50
EDGE_LENGTH: float = 0.95
NORMAL_ANGLE: float = 5.0

ESTIMATE_SCALE: bool = True

# RANSAC parameters
SAMPLE_COUNT: int = 3
MAX_ITERATIONS: int = 200000
CONFIDENCE: float = 0.99999  # NOTE: 0.999


def downsample_and_estimate_normals(cloud: PointCloud, voxel_size: float) -> PointCloud:
    """Preprocesses a point cloud by downsampling and estimating the normals."""
    downsampled: PointCloud = downsample_point_cloud(cloud, spacing=voxel_size)
    preprocessed: PointCloud = estimate_point_cloud_normals(downsampled)
    return preprocessed


def registration_worker(
    source_loader: PointCloudLoader,
    target_loader: PointCloudLoader,
    preprocessor: Callable[[PointCloud], PointCloud],
    registrator: Callable[[PointCloud, PointCloud], ExtendedRegistrationResult],
) -> ExtendedRegistrationResult:
    """Worker function that performs the point cloud registration."""

    source_cloud: PointCloud = source_loader().unwrap()
    target_cloud: PointCloud = target_loader().unwrap()

    preprocessed_source: PointCloud = preprocessor(source_cloud)
    preprocessed_target: PointCloud = preprocessor(target_cloud)

    result: ExtendedRegistratorResult = registrator(
        source=preprocessed_source, target=preprocessed_target
    )

    return result


preprocessor: Callable[[PointCloud], PointCloud] = partial(
    downsample_and_estimate_normals, voxel_size=VOXEL_SIZE
)

registrator = partial(
    register_point_cloud_fphp_ransac,
    distance_threshold=CORRESPONDENCE_DISTANCE,
    feature_radius=FEATURE_RADIUS,
    feature_neighbours=FEATURE_NEIGHBOURS,
    max_iterations=MAX_ITERATIONS,
    confidence=CONFIDENCE,
    sample_count=SAMPLE_COUNT,
    edge_check=EDGE_LENGTH,
    normal_check=NORMAL_ANGLE,
    scaling=ESTIMATE_SCALE,
)

### Define visualization helpers

In [19]:
def visualize_registration(
    source: PointCloud,
    target: PointCloud,
    transformation: np.ndarray,
    source_color: list = None,
    target_color: list = None,
    title: str = "",
    window_width: int = 1024,
    window_height: int = 768,
) -> None:
    """TODO"""
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)

    if source_color:
        source_temp.paint_uniform_color(source_color)
    if target_color:
        target_temp.paint_uniform_color(target_color)

    source_temp.transform(transformation)

    vis.draw_geometries(
        geometry_list=[source_temp, target_temp],
        window_name=title,
        width=window_width,
        height=window_height,
    )


def batch_visualize_registration(
    storage: dict[int, dict],
    source_color: list = [0.60, 0.20, 0.20],
    target_color: list = [0.20, 0.20, 0.60],
) -> None:
    """TODO"""
    for source, registrations in storage.items():

        source_cloud: PointCloud = loaders[source]().unwrap()

        for target, result in registrations.items():

            target_cloud: PointCloud = loaders[target]().unwrap()

            visualize_registration(
                source=source_cloud,
                target=target_cloud,
                transformation=result.transformation,
                source_color=source_color,
                target_color=target_color,
                title=f"Source: {source}, target: {target}",
            )

In [20]:
def log_registration(source, target, result, elapsed: float = 0.0) -> None:
    logger.info("")
    logger.info(f"--------------------- Registration --------------------")
    logger.info(f" - Source, target:        {source}, {target}")
    logger.info(f" - Elapsed time:          {elapsed}")
    logger.info(f" - RMSE:                  {result.inlier_rmse}")
    logger.info(f" - Fitness:               {result.fitness}")
    logger.info(f" - Correspondences:       {len(result.correspondence_set)}")
    logger.info(f" - Transformation:        {result.transformation}")
    logger.info(f"-------------------------------------------------------")
    logger.info("")

### Test registrator parameters on a single case

In [30]:
# Select some point clouds to tune the registration - 0-3 is the really hard case, but 1-3 and 2-3 are also challenging
test_source: int = 0
test_target: int = 3


with util.VerbosityContextManager(util.VerbosityLevel.Debug) as cm:
    source_cloud: PointCloud = loaders[test_source]().unwrap()
    target_cloud: PointCloud = loaders[test_target]().unwrap()

for i in range(10):
    test_result: ExtendedRegistrationResult = registration_worker(
        source_loader=lambda: Ok(source_cloud),
        target_loader=lambda: Ok(target_cloud),
        preprocessor=preprocessor,
        registrator=registrator,
    )

    logger.info(test_result)

[Open3D DEBUG] Format auto File /home/martin/dev/benthoscan/.cache/qdc5ghs3_20100430_024508.ply
[Open3D DEBUG] Read geometry::PointCloud: 10457378 vertices.
[Open3D DEBUG] Format auto File /home/martin/dev/benthoscan/.cache/qdc5ghs3_20210315_230947.ply
[Open3D DEBUG] Read geometry::PointCloud: 15048357 vertices.


2024-07-05 09:48:21.587 | INFO     | __main__:<module>:18 - ExtendedRegistrationResult(fitness=0.5359447004608295, inlier_rmse=0.2064619749672938, correspondence_set=std::vector<Eigen::Vector2i> with 6978 elements.
Use numpy.asarray() to access data., transformation=array([[ 0.33649709, -0.90175887,  0.00851747,  6.031488  ],
       [ 0.90175178,  0.33637131, -0.01303672,  2.2467073 ],
       [ 0.00923702,  0.01253718,  0.96240812, -1.5832695 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]]), information=array([[ 1.56277329e+07, -2.01494085e+05,  3.52026878e+05,
         0.00000000e+00,  3.23495356e+05,  4.78870788e+04],
       [-2.01494085e+05,  1.53372947e+07,  2.22434873e+06,
        -3.23495356e+05,  0.00000000e+00, -8.31615040e+03],
       [ 3.52026878e+05,  2.22434873e+06,  9.55065361e+05,
        -4.78870788e+04,  8.31615040e+03,  0.00000000e+00],
       [ 0.00000000e+00, -3.23495356e+05, -4.78870788e+04,
         6.97800000e+03,  0.00000000e+00,  0.00000000e+00],

In [10]:
visualize_test: bool = True

if visualize_test:
    visualize_registration(
        source=loaders[test_source]().unwrap(),
        target=loaders[test_target]().unwrap(),
        transformation=test_result.transformation,
        title=f"Test case: {test_source}, {test_target}",
    )

### Generate indices and perform registration

In [11]:
indices: list[MultiTargetIndex] = generate_cascade_indices(len(loaders))

result_storage = {
    "feature_matching": dict(),
    "incremental_coarse": dict(),
}

In [12]:
result_storage: dict[int, dict] = dict()

for index in indices:
    source = index.source

    results: dict[int, ExtendedRegistrationResult] = dict()

    for target in index.targets:
        start: float = time()

        result: ExtendedRegistrationResult = registration_worker(
            source_loader=loaders[source],
            target_loader=loaders[target],
            preprocessor=preprocessor,
            registrator=registrator,
        )

        end: float = time()

        elapsed: float = end - start

        log_registration(source, target, result, elapsed)

        results[target] = result

    result_storage["feature_matching"][source] = results

2024-07-05 09:24:20.673 | INFO     | __main__:log_registration:2 - 
2024-07-05 09:24:20.674 | INFO     | __main__:log_registration:3 - --------------------- Registration --------------------
2024-07-05 09:24:20.674 | INFO     | __main__:log_registration:4 -  - Source, target:        0, 1
2024-07-05 09:24:20.675 | INFO     | __main__:log_registration:5 -  - Elapsed time:          27.442278146743774
2024-07-05 09:24:20.675 | INFO     | __main__:log_registration:6 -  - RMSE:                  0.1126804954213844
2024-07-05 09:24:20.675 | INFO     | __main__:log_registration:7 -  - Fitness:               0.8621570966093031
2024-07-05 09:24:20.676 | INFO     | __main__:log_registration:8 -  - Correspondences:       25300
2024-07-05 09:24:20.676 | INFO     | __main__:log_registration:9 -  - Transformation:        [[ 9.93570476e-01 -2.81444000e-02  1.96705283e-03 -1.22882283e+01]
 [ 2.81503568e-02  9.93567572e-01 -3.05038280e-03 -8.68091413e+00]
 [-1.87988259e-03  3.10486288e-03  9.93964332e-01

KeyboardInterrupt: 

### Draw registration and plot results
- TODO: Draw registered point clouds
- TODO: Plot point clouds, correspondences, and error distribution

In [ ]:
batch_visualize_registration(storage=result_storage["feature_matching"])

### Perform coarse incremental registration

In [ ]:
icp_coarse_parameters = dict()
icp_coarse_parameters["voxel"] = 0.05
icp_coarse_parameters["distance"] = 0.075

loss = reg.TukeyLoss(k=0.20)
icp_coarse_parameters["estimator"] = regi.TransformationEstimationPointToPlane(loss)

icp_preprocessor: Callable[[PointCloud], PointCloud] = partial(
    downsample_point_cloud,
    spacing=icp_coarse_parameters["voxel"],
)

icp_registrator = partial(
    register_point_cloud_icp,
    distance_threshold=icp_coarse_parameters["distance"],
    distance_measure=icp_coarse_parameters["estimator"],
)

In [ ]:
result_storage["incremental_coarse"] = dict()


for index in indices:
    source = index.source

    results: dict[int, ExtendedRegistrationResult] = dict()

    for target in index.targets:

        transformation: np.ndarray = result_storage["feature_matching"][source][
            target
        ].transformation

        source_cloud: PointCloud = loaders[source]().unwrap()
        target_cloud: PointCloud = loaders[target]().unwrap()

        source_prepped: PointCloud = icp_preprocessor(source_cloud)
        target_prepped: PointCloud = icp_preprocessor(target_cloud)

        result: ExtendedRegistrationResult = icp_registrator(
            source=source_prepped, target=target_prepped, transformation=transformation
        )

        logger.info("")
        logger.info(f"Source {source}, target {target}")
        logger.info(f"Initial transformation: {transformation}")
        logger.info(f"Refined transformation: {result.transformation}")
        logger.info("")

        results[target] = result

    result_storage["incremental_coarse"][source] = results

In [ ]:
batch_visualize_registration(
    storage=result_storage["incremental_coarse"],
    source_color=[0.60, 0.20, 0.20],
    target_color=[0.20, 0.20, 0.60],
)

### TODO: Perform fine / colored registration

In [ ]:
# TODO: Add colored ICP

### TODO: Perform multiway registration

In [ ]:
def build_pose_graph(
    results: dict[int, dict[int, ExtendedRegistrationResult]],
) -> reg.PoseGraph:
    """Builds a pose graph."""

    odometry = np.identity(4)

    pose_graph = reg.PoseGraph()
    pose_graph.nodes.append(reg.PoseGraphNode(odometry))

    for source_id, registrations in results.items():

        for target_id, result in registrations.items():

            if target_id == source_id + 1:  # odometry case

                odometry = np.dot(result.transformation, odometry)

                pose_graph.nodes.append(
                    reg.PoseGraphNode(
                        np.linalg.inv(odometry),
                    )
                )
                pose_graph.edges.append(
                    reg.PoseGraphEdge(
                        source_id,
                        target_id,
                        result.transformation,
                        result.information,
                        uncertain=False,  # NOTE: False
                    )
                )
            else:  # loop closure case
                pose_graph.edges.append(
                    reg.PoseGraphEdge(
                        source_id,
                        target_id,
                        result.transformation,
                        result.information,
                        uncertain=True,
                    )
                )

    return pose_graph


def optimize_pose_graph(
    graph: reg.PoseGraph,
    correspondence_distance: float,
    prune_threshold: float,
    preference_loop_closure: float,
    reference_node: int = -1,
) -> reg.PoseGraph:
    """TODO"""

    option = reg.GlobalOptimizationOption(
        max_correspondence_distance=correspondence_distance,
        edge_prune_threshold=prune_threshold,
        preference_loop_closure=preference_loop_closure,
        reference_node=reference_node,
    )

    reg.global_optimization(
        graph,
        reg.GlobalOptimizationLevenbergMarquardt(),
        reg.GlobalOptimizationConvergenceCriteria(),
        option,
    )

    return graph

In [ ]:
initial_graph: reg.PoseGraph = build_pose_graph(result_storage["incremental_coarse"])

with util.VerbosityContextManager(util.VerbosityLevel.Debug) as cm:
    optimized_graph: reg.PoseGraph = optimize_pose_graph(
        initial_graph,
        correspondence_distance=0.075,
        prune_threshold=0.25,
        preference_loop_closure=10.0,
        reference_node=0,
    )

for identifier in result_storage["incremental_coarse"]:
    logger.info(optimized_graph.nodes[identifier].pose)

In [ ]:
transformed_clouds: list[PointCloud] = list()

for identifier in result_storage["incremental_coarse"]:

    cloud: PointCloud = loaders[identifier]().unwrap()
    cloud.transform(optimized_graph.nodes[identifier].pose)
    transformed_clouds.append(cloud)

vis.draw_geometries(transformed_clouds)

### TODO: Draw final registration results